In [1]:
import sys
sys.path.insert(0,"../../")

from library import *
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import requests
from bs4 import *
from bs4.element import PageElement


# Get Page Content

In [2]:
def new_session(headers=None):
    session = requests.Session()
    if headers:
        session.headers.update(headers)
    return session

def get_url(url):
    content = new_session().get(url).text
    return content

def strip_html_comment(string): 
    import re
    return re.sub("<!--|-->","",string)


In [3]:
page_content = strip_html_comment(get_url('https://www.basketball-reference.com/boxscores/202110190LAL.html'))
soup = BeautifulSoup(page_content)
str(soup)[:2000]

'<!DOCTYPE html>\n<html class="no-js" data-root="/home/bbr/build" data-version="klecko-" itemscope="" itemtype="https://schema.org/WebSite" lang="en">\n<head>\n<meta charset="utf-8"/>\n<meta content="ie=edge" http-equiv="x-ua-compatible"/>\n<meta content="width=device-width, initial-scale=1.0, maximum-scale=2.0" name="viewport"/>\n<link href="https://d2p3bygnnzw9w3.cloudfront.net/req/202204281" rel="dns-prefetch"/></head><body><p>\n     Quantcast Choice. Consent Manager Tag v2.0 (for TCF 2.0) \n<script async="true" type="text/javascript">\n    (function() {\n\tvar host = window.location.hostname;\n\tvar element = document.createElement(\'script\');\n\tvar firstScript = document.getElementsByTagName(\'script\')[0];\n\tvar url = \'https://quantcast.mgr.consensu.org\'\n\t    .concat(\'/choice/\', \'XwNYEpNeFfhfr\', \'/\', host, \'/choice.js\')\n\tvar uspTries = 0;\n\tvar uspTriesLimit = 3;\n\telement.async = true;\n\telement.type = \'text/javascript\';\n\telement.src = url;\n\t\n\tfirstSc

# Box Scores

In [4]:
def parse_table(source, trow_selector):
    soup = source if isinstance(source,(BeautifulSoup,PageElement)) else BeautifulSoup(source)
    row_tags = soup.select(trow_selector)
    parsed_rows = [parse_trow(row) for row in row_tags]
    return pd.DataFrame(parsed_rows)

def parse_trow(source):
    soup = source if isinstance(source,(BeautifulSoup,PageElement)) else BeautifulSoup(source)
    return {ele["data-stat"]:ele.text for ele in soup.select('th,td')}     # values = soup.get_text(sep).split(sep)

def parse_box_scores(source,teams):
    away,home = teams
    box_scores_meta = {
        'four_factors'  : f'#four_factors > tbody > tr',
        'line_score'    : f'#line_score > tbody > tr',
        'away-basic'    : f'#box-{away}-game-basic > tbody > tr:not(.thead)',
        'away-advanced' : f'#box-{away}-game-advanced > tbody > tr:not(.thead)',
        'home-basic'    : f'#box-{home}-game-basic > tbody > tr:not(.thead)',
        'home-advanced' : f'#box-{home}-game-advanced > tbody > tr:not(.thead)',
        # 'away-basic'    : f'#box-{away}-game-basic > tfoot > tr:not(.thead)',
        **{
            f'away-{period}-basic': f'#box-{away}-{period}-basic > tbody > tr:not(.thead)'
                for period in ['q1','q2','q3','q4','h1','h2']
        },
        **{
            f'home-{period}-basic': f'#box-{home}-{period}-basic > tbody > tr:not(.thead)'
                for period in ['q1','q2','q3','q4','h1','h2']
        },          
    }
    tables = {  
        name : parse_table(source,selector) for name,selector in box_scores_meta.items() 
    }
    return tables

tables = parse_box_scores(soup,('GSW','LAL'))
iter_k = iter(tables.items())
tables['home-basic']

,player,mp,fg,fga,fg_pct,fg3,fg3a,fg3_pct,ft,fta,...,drb,trb,ast,stl,blk,tov,pf,pts,plus_minus,reason
0,Anthony Davis,38:55,15,26,.577,1,5,.200,2,7,...,9,11,2,1,2,0,0,33,-2,NaN
1,LeBron James,36:44,13,23,.565,5,11,.455,3,6,...,10,11,5,1,1,4,5,34,-2,NaN
2,Russell Westbrook,35:08,4,13,.308,0,4,.000,0,0,...,4,5,4,0,0,4,4,8,-23,NaN
3,Kent Bazemore,30:37,3,9,.333,2,8,.250,0,0,...,2,2,0,0,1,1,4,8,+10,NaN
4,DeAndre Jordan,12:55,1,1,1.000,0,0,,0,0,...,2,2,1,1,0,0,3,2,+2,NaN
5,Carmelo Anthony,26:04,3,9,.333,2,4,.500,1,2,...,4,4,2,0,0,1,2,9,-5,NaN
6,Rajon Rondo,19:49,1,4,.250,1,3,.333,0,0,...,0,0,5,2,0,3,2,3,+1,NaN
7,Malik Monk,18:44,2,5,.400,2,4,.500,0,0,...,3,3,1,2,0,1,1,6,-10,NaN
8,Dwight Howard,12:49,1,2,.500,0,0,,3,4,...,6,6,0,0,0,2,2,5,-7,NaN
9,Avery Bradley,8:14,2,3,.667,2,3,.667,0,0,...,0,1,1,0,0,1,2,6,+1,NaN


In [5]:
k,v = next(iter_k)
print(k)
tables[k]

four_factors


,team_id,pace,efg_pct,tov_pct,orb_pct,ft_rate,off_rtg
0,GSW,112.8,.516,13.8,18.4,.269,107.3
1,LAL,112.8,.553,14.1,10.9,.095,101.1


# Fetch Box Scores

In [6]:
season = pd.read_csv("../../data/21-22/results.csv")
season

,Unnamed: 0,Date,Start,Visitor,PTS-V,Home,PTS-H,Matchup,OT,Attend.,Winner,Outcome_HV,Outcome_MU,Score_MU,Code
0,0,2021-10-19,19.5,BRK,104,MIL,127,BRK-MIL,0,17341.0,MIL,Home,1,"(104, 127)",202110190MIL
1,1,2021-10-19,22.0,GSW,121,LAL,114,GSW-LAL,0,18997.0,GSW,Visitor,0,"(121, 114)",202110190LAL
2,2,2021-10-20,19.0,IND,122,CHO,123,CHO-IND,0,15521.0,CHO,Home,0,"(123, 122)",202110200CHO
3,3,2021-10-20,19.0,CHI,94,DET,88,CHI-DET,0,20088.0,CHI,Visitor,0,"(94, 88)",202110200DET
4,4,2021-10-20,19.5,BOS,134,NYK,138,BOS-NYK,2,19812.0,NYK,Home,1,"(134, 138)",202110200NYK
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1225,1225,2022-04-10,21.5,LAL,146,DEN,141,DEN-LAL,1,19520.0,LAL,Visitor,1,"(141, 146)",202204100DEN
1226,1226,2022-04-10,21.5,OKC,88,LAC,138,LAC-OKC,0,18210.0,LAC,Home,0,"(138, 88)",202204100LAC
1227,1227,2022-04-10,21.5,GSW,128,NOP,107,GSW-NOP,0,16595.0,GSW,Visitor,0,"(128, 107)",202204100NOP
1228,1228,2022-04-10,21.5,SAC,116,PHO,109,PHO-SAC,0,17071.0,SAC,Visitor,1,"(109, 116)",202204100PHO


In [8]:
# k,v = next(iter_k)
# print(k)
# tables[k]

In [13]:
def get_box_scores(match_code,write_to_folder=None):
    print(match_code)
    page_content = get_url(f'https://www.basketball-reference.com/boxscores/{match_code}.html')
    source = BeautifulSoup(strip_html_comment(str(page_content)))
    tables = parse_box_scores(source,(d['Visitor'],d['Home']))
    for name,table in tables.items():
        assert(not table.empty)
        if write_to_folder:
            write_dataframe(table,f"{write_to_folder}/{match_code}/{name}.csv")
    return tables

for i,d in season.iterrows():
    get_box_scores(d['Code'],write_to_folder='../../data/21-22/box_scores')

202110190MIL
202110190LAL
202110200CHO
202110200DET
202110200NYK
202110200TOR
202110200MEM
202110200MIN
202110200NOP
202110200SAS
202110200UTA
202110200POR
202110200PHO
202110210ATL
202110210MIA
202110210GSW
202110220ORL
202110220WAS
202110220CLE
202110220BOS
202110220PHI
202110220HOU
202110220CHI
202110220DEN
202110220LAL
202110220SAC
202110230CLE
202110230IND
202110230TOR
202110230CHI
202110230MIN
202110230SAS
202110230POR
202110230LAC
202110240BRK
202110240HOU
202110240NYK
202110240OKC
202110240SAC
202110240LAL
202110250CHO
202110250IND
202110250TOR
202110250MIA
202110250BRK
202110250ATL
202110250MIN
202110250DEN
202110250LAC
202110260NYK
202110260OKC
202110260SAS
202110260DAL
202110260UTA
202110270ORL
202110270BOS
202110270BRK
202110270TOR
202110270NOP
202110270MIL
202110270OKC
202110270PHO
202110270POR
202110270LAC
202110280WAS
202110280PHI
202110280HOU
202110280CHI
202110280DAL
202110280GSW
202110290TOR
202110290MIA
202110290BRK
202110290NOP
202110290DEN
202110290POR
202110290LAL

# Play By Play

In [ ]:
match_code = '202110310BRK'
f'https://www.basketball-reference.com/boxscores/pbp/{match_code}.html'

# Plus Minus

In [ ]:
'https://www.basketball-reference.com/boxscores/plus-minus/202110310BRK.html'

# Shooting Stats

In [ ]:
'https://www.basketball-reference.com/boxscores/shot-chart/202110310BRK.html'